# Import libraries and preprocessing data/images

In [ ]:
!pip install optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import optuna
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
from google.colab import drive
import os

# First, check where you currently are
print("Current directory:", os.getcwd())

# Check GPU
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("Running on CPU")

# Choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

#import cleaned data storred in the previous notebook (01_Data_cleaning_inspection)
drive.mount('/content/drive', force_remount=True)

#preprocess
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])

#apply transform to images
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/train/', transform = transform)
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/data/test/', transform = transform)

# Split training into train/validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

# Data loaders
def get_loaders(batch_size):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

print(train_dataset)

# Define CNN Model

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, conv_filters=32, dense_units=300, dropout_rate=0.3):
        super(CNNModel, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(1, conv_filters, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(conv_filters)

        self.conv2 = nn.Conv2d(conv_filters, conv_filters * 2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(conv_filters * 2)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)

        # Fully connected layers (fc1 created dynamically)
        self.dense_units = dense_units
        self._fc1 = None
        self.fc2 = nn.Linear(dense_units, 2)

    def _create_fc1(self, x):
        """Dynamically create fc1 based on input shape"""
        if self._fc1 is None:
            with torch.no_grad():
                x = self.pool(F.relu(self.bn1(self.conv1(x))))
                x = self.pool(F.relu(self.bn2(self.conv2(x))))
                x = torch.flatten(x, 1)
                num_features = x.shape[1]
            self._fc1 = nn.Linear(num_features, self.dense_units).to(x.device)
            print(f"Created fc1 with input features: {num_features}")

    def forward(self, x):
        self._create_fc1(x)  # Ensure fc1 is created dynamically

        # Convolutional layers with BatchNorm and ReLU
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))

        # Flatten and pass through FC layers
        x = torch.flatten(x, 1)
        x = F.relu(self._fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


# Define Training & Validation Functions

In [ ]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    return avg_loss


def validation(model, device, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct / len(val_loader.dataset)
    return avg_val_loss, accuracy


# Define Objective Function for Optuna

In [ ]:

def objective(trial):
    #Hyperparameters to tune
    conv_filters = trial.suggest_categorical('conv_filters', [16, 32, 64])
    dense_units = trial.suggest_categorical('dense_units', [64, 128, 256])
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [64, 128])

    #Load data
    train_loader, val_loader, test_loader = get_loaders(batch_size)

    #Model, optimizer, and loss
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CNNModel(conv_filters, dense_units, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    #Training and validation
    for epoch in range(5):
        train_loss = train(model, device, train_loader, optimizer, criterion)
        val_loss, val_acc = validation(model, device, val_loader, criterion)

        # Report progress to Optuna
        trial.report(val_acc, epoch)

        # Early pruning if not promising
        if trial.should_prune():
            raise optuna.TrialPruned()

        print(f"Epoch {epoch+1}/5 | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    return val_acc

# Run Hyperparameter Optimization

In [ ]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 5)
print("Best hyperparameters", study.best_params)

# Train Final Model with Best Hyperparameters

In [ ]:
best_params = {'conv_filters': 32, 'dense_units': 64, 'dropout_rate': 0.3075665173910642, 'lr': 0.0006482971515191023, 'batch_size': 64}
print (best_params['batch_size'])

#Use the best hyperparameters from Optuna
#best_params = study.best_params

#Load data using the best batch size
train_loader, val_loader, test_loader = get_loaders(best_params['batch_size'])

#Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Build model with best parameters
model = CNNModel(
    conv_filters=best_params['conv_filters'],
    dense_units=best_params['dense_units'],
    dropout_rate=best_params['dropout_rate']
).to(device)

#Optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'])
criterion = nn.CrossEntropyLoss()

#Final training with best configuration
for epoch in range(10):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    val_loss, val_acc = validation(model, device, val_loader, criterion)
    print(f"Epoch {epoch+1}/10 | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

print("\n ✅ Final training completed.")

# Save Trained Model in Different Methods

In [ ]:
#Save Only the Model Weights
torch.save(model.state_dict(),"/content/drive/MyDrive/model_weights.pth")
#Save the Entire Model (including architecture)
torch.save(model,"/content/drive/MyDrive/full_model.pth")
#Save the Model + Optimizer (for resuming training)
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': train_loss,
    'loss2': val_loss,
},"/content/drive/MyDrive/checkpoint.pth")
#Saving Best Model
torch.save(model.state_dict(),"/content/drive/MyDrive/best_model.pth")
print("Saved best model!")

#Evaluate the Trained Model on Test Set

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

model = torch.load("/content/drive/MyDrive/full_model.pth", weights_only=False)
model.eval()

#Load data using the best batch size = 64 (based on training the model)
train_loader, val_loader, test_loader = get_loaders(64)

criterion = nn.CrossEntropyLoss()

# -----------------------------------------------------
# Collect predictions and true labels from test set
# -----------------------------------------------------

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convert lists to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# -----------------------------------------------------
# Compute Metrics
# -----------------------------------------------------

acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds)
rec = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print("Model Evaluation Results:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")

# Full classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=['Non-Defective', 'Defective']))

# -----------------------------------------------------
# Confusion Matrix
# -----------------------------------------------------

cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Non-Defective', 'Defective'],
            yticklabels=['Non-Defective', 'Defective'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
